Feature Engineering 

In [31]:
import pandas as pd
import numpy as np 
data = pd.read_csv('data/raw/BankChurners.csv')

In [32]:
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.pyplot as plt 

In [46]:
from sklearn.preprocessing import MinMaxScaler

나이 

In [33]:
# 나이(Customer_Age)를 5개 구간으로 나누어 새로운 범주형 변수 생성
bins = [20, 30, 40, 50, 60, 70]
labels = ['20대', '30대', '40대', '50대', '60대+']
data['Age_Group'] = pd.cut(data['Customer_Age'], bins=bins, labels=labels, right=False)

# 결과 확인 (상위 5개 행)
# | Customer_Age | Age_Group |
# |:-------------|:----------|
# | 45           | 40대       |
# | 49           | 40대       |
# | 51           | 50대       |
# | 40           | 40대       |
# | 40           | 40대       |

성별 변수 

In [34]:
# 성별(Gender) 변수를 One-Hot Encoding으로 변환
# drop_first=True로 설정하여 다중공선성을 방지합니다 (Gender_F는 Gender_M으로 설명 가능).
data_encoded = pd.get_dummies(data, columns=['Gender'], prefix='Gender', drop_first=True)

# 결과 확인: Gender_M 열이 추가됨
# | Gender_M |
# |:---------|
# | 1        | (Male)
# | 0        | (Female)
# | 1        | (Male)
# | 0        | (Female)
# | 1        | (Male)

In [35]:
# 'Education_Level', 'Marital_Status', 'Income_Category' 등 
# 모든 범주형 변수에 대해 One-Hot Encoding을 적용해야 합니다.

# drop_first=True는 다중공선성을 방지하기 위해 첫 번째 범주를 제거합니다.
categorical_cols = ['Gender', 'Marital_Status', 'Education_Level', 'Income_Category', 'Card_Category']
df_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# 'Unknown' 범주는 자동으로 'Marital_Status_Unknown' 등의 새로운 열로 인코딩됩니다.

print("--- One-Hot Encoding 결과 (Gender, Marital_Status 열 변화) ---")
print(data_encoded[[col for col in data_encoded.columns if 'Gender_' in col or 'Marital_Status_' in col]].head().to_markdown(index=False))

--- One-Hot Encoding 결과 (Gender, Marital_Status 열 변화) ---
|   Gender_M |
|-----------:|
|          1 |
|          0 |
|          1 |
|          0 |
|          1 |


In [36]:
data['Log_Total_Trans_Amt'] = np.log1p(data['Total_Trans_Amt'])

print("--- 로그 변환 결과 (Log_Total_Trans_Amt 열 추가) ---")
print(data[['Total_Trans_Amt', 'Log_Total_Trans_Amt']].head().to_markdown(index=False, floatfmt=".4f"))

--- 로그 변환 결과 (Log_Total_Trans_Amt 열 추가) ---
|   Total_Trans_Amt |   Log_Total_Trans_Amt |
|------------------:|----------------------:|
|         1144.0000 |                7.0432 |
|         1291.0000 |                7.1639 |
|         1887.0000 |                7.5433 |
|         1171.0000 |                7.0665 |
|          816.0000 |                6.7056 |


In [ ]:
# pip install tabulate 

Note: you may need to restart the kernel to use updated packages.


고객활동점수 + Risk Score

In [30]:
# 고객활동점수율 기반 Feature

data["Activity_Score"] = (
    data["Total_Trans_Ct"] * 0.6 +
    data["Contacts_Count_12_mon"] * (-0.3) +
    data["Months_Inactive_12_mon"] * (-0.5)
)

#전액 사용율 기반 Feature
data["Risk_Score"] = (
    data["Total_Revolving_Bal"] * 0.4 +
    data["Avg_Utilization_Ratio"] * 0.6
)

In [43]:
# 로그 변환 (왜도 높은 컬럼들 처리)
log_cols = ["Total_Trans_Amt", "Total_Revolving_Bal"]
for col in log_cols:
    if col in data.columns:
        data[f"log_{col}"] = np.log1p(data[col])

In [44]:
# 상호작용 변수 생성
data["Activity_Index"] = data["Total_Trans_Amt"] * data["Total_Trans_Ct"]
data["Amt_per_Contact"] = data["Total_Trans_Amt"] / (data["Contacts_Count_12_mon"] + 1)

# 위험도 기반 
data["Risk_Score"] = (
    data["Avg_Utilization_Ratio"] * 0.6 +
    data["Total_Revolving_Bal"] * 0.4
)

#원핫 인코딩
data = pd.get_dummies(data, drop_first=True)

In [ ]:
# 기본 스캐일링
scaler = MinMaxScaler()
numeric_cols = data.select_dtypes(include=["int64", "float64"]).columns

data[numeric_cols] = scaler.fit_transform(data[numeric_cols])